<a href="https://colab.research.google.com/github/dudller/dw_3/blob/master/Day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Flatten, MaxPool2D, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import os
import datetime
import matplotlib.pyplot as plt
from skimage import color,exposure
from sklearn.metrics import accuracy_score
%load_ext tensorboard

from hyperopt import  hp,STATUS_OK,tpe,Trials,fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/part3/dw_3'

/content/drive/My Drive/Colab Notebooks/dw_matrix/part3/dw_3


In [0]:
train=pd.read_pickle('data/train.p')
X_train,y_train = train['features'],train['labels']
test=pd.read_pickle('data/test.p')
X_test,y_test = test['features'],test['labels']

names=pd.read_csv('data/names.csv')

In [0]:
y_train,y_test=to_categorical(y_train),to_categorical(y_test)

In [0]:
input_shape=X_train.shape[1:]
output=y_train.shape[1]

In [0]:
def train_model(X_train,y_train,model,fit_params={}):
  model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
  logdir=os.path.join('logs',datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback=tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)
  model.fit(X_train,y_train,
            batch_size=fit_params.get('batch_size',128),
            epochs=fit_params.get('epochs',5),
            verbose=fit_params.get('verbose',1),
            validation_data=fit_params.get('val_data',(X_train,y_train)),
            callbacks=[tensorboard_callback]
            )
  return model

def predict_and_score(model,X_test,y_test,scoring=accuracy_score):
  y_test_norm=np.argmax(y_test,axis=1)
  pred=model.predict(X_test)
  pred_norm=np.argmax(pred,axis=1)
  return scoring(y_test_norm,pred_norm)

def train_and_pred(model):
  T_model=train_model(X_train,y_train,model2,params)
  return predict_and_score(T_model,X_test,y_test)

In [0]:
names = names.to_dict()['b']


In [0]:
def get_cnn_6(input_shape,output):
  return Sequential([
       Conv2D(filters=32,kernel_size=(3,3),input_shape=input_shape,activation='relu',padding='same'),
       Conv2D(filters=32,kernel_size=(3,3),activation='relu'),
       MaxPool2D(),
       Dropout(0.3),
       Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
       Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
       MaxPool2D(),
       Dropout(0.35),
       Conv2D(filters=124,kernel_size=(3,3),activation='relu',padding='same'),
       Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
       MaxPool2D(),
       Dropout(0.35),
       Flatten(),
       Dense(1024,activation='relu'),
       Dropout(0.3),
       Dense(512,activation='relu'),
       Dropout(0.25),
       Dense(512,activation='relu'),
       Dropout(0.3),

       Dense(output,activation='softmax')           
        ])

In [11]:
model7=get_cnn_6(input_shape,output)
T_model7=train_model(X_train,y_train,model7)
predict_and_score(T_model7,X_test,y_test)

Epoch 1/5
272/272 [==============================] - 5s 18ms/step - loss: 2.7987 - accuracy: 0.2233 - val_loss: 1.4463 - val_accuracy: 0.5459
Epoch 2/5
272/272 [==============================] - 4s 15ms/step - loss: 1.2110 - accuracy: 0.6111 - val_loss: 0.4330 - val_accuracy: 0.8580
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.5966 - accuracy: 0.8104 - val_loss: 0.1583 - val_accuracy: 0.9532
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.3829 - accuracy: 0.8826 - val_loss: 0.1091 - val_accuracy: 0.9656
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2810 - accuracy: 0.9157 - val_loss: 0.0477 - val_accuracy: 0.9874


0.9643990929705215

In [12]:
T_model7.evaluate(X_test,y_test)

138/138 [==============================] - 0s 4ms/step - loss: 0.1524 - accuracy: 0.9644


[0.1524198055267334, 0.9643990993499756]

In [19]:
def get_model(params):
  return Sequential([
       Conv2D(filters=32,kernel_size=(3,3),input_shape=input_shape,activation='relu',padding='same'),
       Conv2D(filters=32,kernel_size=(3,3),activation='relu'),
       MaxPool2D(),
       Dropout(params['block_cnn_1_dropout_1']),
       Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
       Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
       MaxPool2D(),
       Dropout(params['block_cnn_2_dropout_1']),
       Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same'),
       Conv2D(filters=128,kernel_size=(3,3),activation='relu'),
       MaxPool2D(),
       Dropout(params['block_cnn_3_dropout_1']),
       Flatten(),
       Dense(1024,activation='relu'),
       Dropout(params['block_dense_dropout_1']),
       Dense(1024,activation='relu'),
       Dropout(params['block_dense_dropout_2']),

       Dense(output,activation='softmax')           
        ])

ERROR! Session/line number was not unique in database. History logging moved to new session 60


In [0]:
def func_obj(params):
  model=get_model(params)
  model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
  model.fit(X_train,y_train,
            batch_size=int(params.get('batch_size',128)),
            epochs=params.get('epochs',7),
            verbose=0
            )
  score=model.evaluate(X_test,y_test)
  accuracy=score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss':-accuracy,'status':STATUS_OK,'model':model}


space= {
    'batch_size': hp.quniform('batch_size',100,200,10),
    'block_cnn_1_dropout_1':hp.uniform('block_cnn_1_dropout_1',0.0,0.5),
    'block_cnn_2_dropout_1':hp.uniform('block_cnn_2_dropout_1',0.0,0.5),
    'block_cnn_3_dropout_1':hp.uniform('block_cnn_3_dropout_1',0.0,0.7),
    'block_dense_dropout_1':hp.uniform('block_dense_dropout_1',0.0,0.7),
    'block_dense_dropout_2':hp.uniform('block_dense_dropout_2',0.0,0.7),


}



In [32]:
best=fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 1.7049 - accuracy: 0.5312

 10/138 [=>............................]
 - ETA: 0s - loss: 0.1955 - accuracy: 0.9438

 18/138 [==>...........................]
 - ETA: 0s - loss: 0.2043 - accuracy: 0.9479

 26/138 [====>.........................]
 - ETA: 0s - loss: 0.4477 - accuracy: 0.9219

 35/138 [======>.......................]
 - ETA: 0s - loss: 0.3329 - accuracy: 0.9420

 43/138 [========>.....................]
 - ETA: 0s - loss: 0.2759 - accuracy: 0.9520
